In [0]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("YourAppName") \
    .getOrCreate()

In [0]:
spark.conf.set("spark.databricks.io.cache.enabled", "false")
print("spark.databricks.io.cache.enabled is %s" % spark.conf.get("spark.databricks.io.cache.enabled"))

spark.databricks.io.cache.enabled is false


In [0]:
%pip install modin joblib

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import pandas
import numpy as np

import modin.pandas as pd
import joblib

#import databricks.koalas as ks
# import dask.dataframe as dd
# from dask.distributed import Client, LocalCluster

print('pandas version: %s' % pandas.__version__)

print('numpy version: %s' % np.__version__)

print('modin pandas version: %s' % pd.__version__)

print('joblib version: %s' % joblib.__version__)

#print('koalas version: %s' % ks.__version__)

# import dask
# print('dask version: %s' % dask.__version__)

import pyarrow
print('pyarrow version: %s' % pyarrow.__version__)

import pyspark
print('pyspark version: %s' % pyspark.__version__)


import time

def benchmark(f, df, benchmarks, name, **kwargs):
    """Benchmark the given function against the given DataFrame.
    
    Parameters
    ----------
    f: function to benchmark
    df: data frame
    benchmarks: container for benchmark results
    name: task name
    
    Returns
    -------
    Duration (in seconds) of the given operation
    """
    start_time = time.time()
    ret = f(df, **kwargs)
    benchmarks['duration'].append(time.time() - start_time)
    benchmarks['task'].append(name)
    print(f"{name} took: {benchmarks['duration'][-1]} seconds")
    return benchmarks['duration'][-1]

def get_results(benchmarks):
    """Return a pandas DataFrame containing benchmark results."""
    return pd.DataFrame.from_dict(benchmarks)

pandas version: 2.2.2
numpy version: 1.26.4
modin pandas version: 0.30.0
joblib version: 1.1.1
pyarrow version: 7.0.0
pyspark version: 3.3.2.dev0


In [0]:
import pandas as pd

file_path = "/FileStore/tables/yellow_tripdata_2023_01.parquet"
#pandas_data = pd.read_parquet(file_path, engine="pyarrow")
pandas_data = spark.read.format('parquet').options(header='true').load(file_path).toPandas()

In [0]:
modin_data = pd.DataFrame(pandas_data)
modin_benchmarks = {
    'duration': [],  # in seconds
    'task': [],
}
modin_data.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [0]:
from modin.config import Engine
# Set the engine to 'joblib'
# Engine.put("joblib")

def read_file_parquet():
    return pd.read_parquet("/FileStore/tables/yellow_tripdata_2023_01.parquet")

def count(df):
    return len(df)

def count_index_length(df):
    return len(df.index)

def mean(df):
    return df['fare_amount'].mean()

def standard_deviation(df):
    return df['fare_amount'].std()

def mean_of_sum(df):
    return (df['fare_amount'] + df['tip_amount']).mean()

def sum_columns(df):
    return (df['fare_amount'] + df['tip_amount'])

def mean_of_product(df):
    return (df['fare_amount'] * df['tip_amount']).mean()

def product_columns(df):
    return (df['fare_amount'] * df['tip_amount'])

def value_counts(df):
    return df['fare_amount'].value_counts()

def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df['start_lon']
    phi_1 = df['start_lat']
    theta_2 = df['end_lon']
    phi_2 = df['end_lat']
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2 +
            np.cos(theta_1*np.pi/180) * np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret.mean()

def complicated_arithmetic_operation(df):
    theta_1 = df['start_lon']
    phi_1 = df['start_lat']
    theta_2 = df['end_lon']
    phi_2 = df['end_lat']
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2 +
            np.cos(theta_1*np.pi/180) * np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret

def groupby_statistics(df):
    result = df.groupby('passenger_count').agg(
        fare_amount_mean=('fare_amount', 'mean'),
        fare_amount_std=('fare_amount', 'std'),
        tip_amount_mean=('tip_amount', 'mean'),
        tip_amount_std=('tip_amount', 'std')
    )
    return result

other = groupby_statistics(modin_data)
# other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

def join_count(df, other):
    return len(pd.merge(df, other, left_index=True, right_index=True))

def join_data(df, other):
    return pd.merge(df, other, left_index=True, right_index=True)

In [0]:
#benchmark(read_file_parquet, df=None, benchmarks=dask_benchmarks, name='read file')
benchmark(count, df=modin_data, benchmarks=modin_benchmarks, name='count')
benchmark(count_index_length, df=modin_data, benchmarks=modin_benchmarks, name='count index length')
benchmark(mean, df=modin_data, benchmarks=modin_benchmarks, name='mean')
benchmark(standard_deviation, df=modin_data, benchmarks=modin_benchmarks, name='standard deviation')
benchmark(mean_of_sum, df=modin_data, benchmarks=modin_benchmarks, name='mean of columns addition')
benchmark(sum_columns, df=modin_data, benchmarks=modin_benchmarks, name='addition of columns')
benchmark(mean_of_product, df=modin_data, benchmarks=modin_benchmarks, name='mean of columns multiplication')
benchmark(product_columns, df=modin_data, benchmarks=modin_benchmarks, name='multiplication of columns')
benchmark(value_counts, df=modin_data, benchmarks=modin_benchmarks, name='value counts')
# No column for this
# benchmark(mean_of_complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='mean of complex arithmetic ops')
# benchmark(complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='complex arithmetic ops')
benchmark(groupby_statistics, df=modin_data, benchmarks=modin_benchmarks, name='groupby statistics')
benchmark(join_count, modin_data, benchmarks=modin_benchmarks, name='join count', other=other)
benchmark(join_data, modin_data, benchmarks=modin_benchmarks, name='join', other=other) # cant join

count took: 1.5020370483398438e-05 seconds
count index length took: 5.4836273193359375e-06 seconds
mean took: 0.008271455764770508 seconds
standard deviation took: 0.05388307571411133 seconds
mean of columns addition took: 0.021149635314941406 seconds
addition of columns took: 0.007768869400024414 seconds
mean of columns multiplication took: 0.014803647994995117 seconds
multiplication of columns took: 0.007726430892944336 seconds
value counts took: 0.07363486289978027 seconds
groupby statistics took: 0.18078207969665527 seconds
join count took: 0.03470349311828613 seconds
join took: 0.015247106552124023 seconds
Out[5]: 0.015247106552124023

Operations with filtering

In [0]:
expr_filter = (modin_data.tip_amount >= 1) & (modin_data.tip_amount <= 5)

def filter_data(df):
    return df[expr_filter]
  
modin_filtered = filter_data(modin_data)

In [0]:
benchmark(count, modin_filtered, benchmarks=modin_benchmarks, name='filtered count')
benchmark(count_index_length, modin_filtered, benchmarks=modin_benchmarks, name='filtered count index length')
benchmark(mean, modin_filtered, benchmarks=modin_benchmarks, name='filtered mean')
benchmark(standard_deviation, modin_filtered, benchmarks=modin_benchmarks, name='filtered standard deviation')
benchmark(mean_of_sum, modin_filtered, benchmarks=modin_benchmarks, name ='filtered mean of columns addition')
benchmark(sum_columns, df=modin_filtered, benchmarks=modin_benchmarks, name='filtered addition of columns')
benchmark(mean_of_product, modin_filtered, benchmarks=modin_benchmarks, name ='filtered mean of columns multiplication')
benchmark(product_columns, df=modin_filtered, benchmarks=modin_benchmarks, name='filtered multiplication of columns')
#benchmark(mean_of_complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered mean of complex arithmetic ops')
#benchmark(complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered complex arithmetic ops')
benchmark(value_counts, modin_filtered, benchmarks=modin_benchmarks, name ='filtered value counts')
benchmark(groupby_statistics, modin_filtered, benchmarks=modin_benchmarks, name='filtered groupby statistics')

other = groupby_statistics(modin_filtered)
# other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

benchmark(join_count, modin_filtered, benchmarks=modin_benchmarks, name='filtered join count', other=other)
benchmark(join_data, modin_filtered, benchmarks=modin_benchmarks, name='filtered join', other=other)

filtered count took: 1.2159347534179688e-05 seconds
filtered count index length took: 5.7220458984375e-06 seconds
filtered mean took: 0.005141735076904297 seconds
filtered standard deviation took: 0.028097152709960938 seconds
filtered mean of columns addition took: 0.009072303771972656 seconds
filtered addition of columns took: 0.0043392181396484375 seconds
filtered mean of columns multiplication took: 0.008221149444580078 seconds
filtered multiplication of columns took: 0.004019737243652344 seconds
filtered value counts took: 0.03287196159362793 seconds
filtered groupby statistics took: 0.09776163101196289 seconds
filtered join count took: 0.009285211563110352 seconds
filtered join took: 0.008361339569091797 seconds
Out[7]: 0.008361339569091797

In [0]:
modin_res_temp = get_results(modin_benchmarks).set_index('task')
modin_res_temp

,duration
task,
count,0.000015
count index length,0.000005
mean,0.008271
standard deviation,0.053883
mean of columns addition,0.021150
addition of columns,0.007769
mean of columns multiplication,0.014804
multiplication of columns,0.007726
value counts,0.073635
